<a href="https://colab.research.google.com/github/Kvazzzzar/MPSI/blob/main/MPSI_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import string
import nltk

# Скачиваем необходимые ресурсы NLTK
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

class TextDataProcessor:
    """
    Класс для обработки текстовых данных от сбора до разделения на выборки
    Включает:
    - Загрузку датасета
    - Предобработку текста
    - Векторизацию
    - Кластеризацию
    - Разделение на train/test/val
    """

    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))

    def load_dataset(self, path):
        """Загрузка датасета"""
        self.df = pd.read_csv(path)
        print(f"Загружено {len(self.df)} записей")
        print("Пример данных:")
        print(self.df.head())
        return self.df

    def preprocess_text(self, text):
        """Предобработка текста: токенизация, лемматизация, очистка"""
        # Приведение к нижнему регистру
        text = text.lower()

        # Удаление пунктуации
        text = text.translate(str.maketrans('', '', string.punctuation))

        # Токенизация
        tokens = word_tokenize(text)

        # Удаление стоп-слов и лемматизация
        tokens = [self.lemmatizer.lemmatize(token) for token in tokens
                 if token not in self.stop_words and token.isalpha()]

        return ' '.join(tokens)

    def analyze_data(self):
        """Анализ данных"""
        print("\nАнализ данных:")
        print(f"Количество категорий: {len(self.df['category'].unique())}")
        print("Распределение категорий:")
        print(self.df['category'].value_counts())

        # Длина текстов
        self.df['text_length'] = self.df['text'].apply(lambda x: len(x.split()))
        print("\nСтатистика длины текстов:")
        print(self.df['text_length'].describe())

    def vectorize_text(self):
        """Векторизация текста с помощью TF-IDF"""
        self.vectorizer = TfidfVectorizer(max_features=5000)
        self.X = self.vectorizer.fit_transform(self.df['processed_text'])
        print(f"\nРазмерность матрицы признаков: {self.X.shape}")

    def cluster_texts(self):
        """Кластеризация текстов и сравнение с реальными категориями"""
        n_clusters = len(self.df['category'].unique())
        kmeans = KMeans(n_clusters=n_clusters, random_state=42)
        clusters = kmeans.fit_predict(self.X)

        # Сравнение с реальной разметкой
        ari_score = adjusted_rand_score(self.df['category'], clusters)
        print(f"\nAdjusted Rand Index (сравнение с реальными категориями): {ari_score:.2f}")

        # Добавляем кластеры в датасет для анализа
        self.df['cluster'] = clusters

    def split_data(self):
        """Разделение данных на train/test/val"""
        # Сначала разделяем на train+val и test
        X_temp, X_test, y_temp, y_test = train_test_split(
            self.X, self.df['category'], test_size=0.15, random_state=42, stratify=self.df['category'])

        # Затем разделяем train+val на train и val
        X_train, X_val, y_train, y_val = train_test_split(
            X_temp, y_temp, test_size=0.15/0.85, random_state=42, stratify=y_temp)

        print("\nРазмеры выборок:")
        print(f"Train: {X_train.shape[0]} samples")
        print(f"Val: {X_val.shape[0]} samples")
        print(f"Test: {X_test.shape[0]} samples")

        return X_train, X_val, X_test, y_train, y_val, y_test

    def process_pipeline(self, dataset_path):
        """Полный пайплайн обработки данных"""
        # 1. Collect data
        self.load_dataset(dataset_path)

        # 2. Prepare data
        print("\nПредобработка текстов...")
        self.df['processed_text'] = self.df['text'].apply(self.preprocess_text)

        # 3. Analyze data
        self.analyze_data()

        # 4. Vectorize data
        self.vectorize_text()

        # 5. Cluster data
        self.cluster_texts()

        # 6. Split data
        return self.split_data()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
import tarfile
import requests
import os

# Скачать файл (пример для 20news-bydate.tar.gz)
url = "http://qwone.com/~jason/20Newsgroups/20news-bydate.tar.gz"
response = requests.get(url, stream=True)

with open("20news-bydate.tar.gz", "wb") as f:
    f.write(response.content)

# Распаковать
with tarfile.open("20news-bydate.tar.gz", "r:gz") as tar:
    tar.extractall()

# Проверить структуру
!ls 20news-bydate-train  # Учебные данные
!ls 20news-bydate-test   # Тестовые данные

alt.atheism		  comp.windows.x      rec.sport.hockey	soc.religion.christian
comp.graphics		  misc.forsale	      sci.crypt		talk.politics.guns
comp.os.ms-windows.misc   rec.autos	      sci.electronics	talk.politics.mideast
comp.sys.ibm.pc.hardware  rec.motorcycles     sci.med		talk.politics.misc
comp.sys.mac.hardware	  rec.sport.baseball  sci.space		talk.religion.misc
alt.atheism		  comp.windows.x      rec.sport.hockey	soc.religion.christian
comp.graphics		  misc.forsale	      sci.crypt		talk.politics.guns
comp.os.ms-windows.misc   rec.autos	      sci.electronics	talk.politics.mideast
comp.sys.ibm.pc.hardware  rec.motorcycles     sci.med		talk.politics.misc
comp.sys.mac.hardware	  rec.sport.baseball  sci.space		talk.religion.misc


In [8]:
import pandas as pd
from pathlib import Path

def load_20newsgroups_to_df(folder_path):
    """
    Загружает данные 20 Newsgroups из структуры папок в DataFrame
    """
    data = []
    for category in Path(folder_path).iterdir():
        if category.is_dir():
            for file in category.iterdir():
                with open(file, 'r', encoding='latin1') as f:
                    text = f.read()
                data.append({
                    'text': text,
                    'category': category.name,
                    'source': 'train' if 'train' in str(folder_path) else 'test'
                })
    return pd.DataFrame(data)

# Загрузка train и test данных
print("Загрузка train данных...")
df_train = load_20newsgroups_to_df("20news-bydate-train")
print(f"Загружено {len(df_train)} train записей")

print("\nЗагрузка test данных...")
df_test = load_20newsgroups_to_df("20news-bydate-test")
print(f"Загружено {len(df_test)} test записей")

# Объединение данных
print("\nОбъединение данных...")
full_df = pd.concat([df_train, df_test], ignore_index=True)
print(f"Всего записей: {len(full_df)}")

# Анализ данных
print("\nРаспределение по категориям:")
print(full_df['category'].value_counts())

print("\nРаспределение по источникам:")
print(full_df['source'].value_counts())

# Сохранение в CSV
output_path = "20news-full.csv"
full_df.to_csv(output_path, index=False)
print(f"\nДанные сохранены в {output_path}")

# Пример первых записей
print("\nПервые 3 записи:")
print(full_df.head(3))

Загрузка train данных...
Загружено 11314 train записей

Загрузка test данных...
Загружено 7532 test записей

Объединение данных...
Всего записей: 18846

Распределение по категориям:
category
rec.sport.hockey            999
soc.religion.christian      997
rec.motorcycles             996
rec.sport.baseball          994
sci.crypt                   991
sci.med                     990
rec.autos                   990
comp.windows.x              988
sci.space                   987
comp.os.ms-windows.misc     985
sci.electronics             984
comp.sys.ibm.pc.hardware    982
misc.forsale                975
comp.graphics               973
comp.sys.mac.hardware       963
talk.politics.mideast       940
talk.politics.guns          910
alt.atheism                 799
talk.politics.misc          775
talk.religion.misc          628
Name: count, dtype: int64

Распределение по источникам:
source
train    11314
test      7532
Name: count, dtype: int64

Данные сохранены в 20news-full.csv

Первые 3 запи

In [6]:
class TextDataProcessor20News(TextDataProcessor):
    def __init__(self):
        super().__init__()

    def load_from_folder(self, folder_path):
        """Альтернативный метод загрузки данных из папки 20 Newsgroups"""
        self.df = load_20newsgroups_to_df(folder_path)
        print(f"Загружено {len(self.df)} записей")
        print("Пример данных:")
        print(self.df.head())
        return self.df

In [13]:
import nltk
nltk.download('punkt_tab')

# 1. Инициализируем процессор
processor = TextDataProcessor20News()

# 2. Загружаем данные (используем объединенный CSV)
processor.load_dataset("20news-full.csv")  # Используем файл, который мы создали ранее

# 3. Запускаем полный пайплайн обработки
try:
    X_train, X_val, X_test, y_train, y_val, y_test = processor.process_pipeline("20news-full.csv")
    print("\nДанные успешно обработаны и разделены!")
except Exception as e:
    print(f"\nПроизошла ошибка: {str(e)}")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Загружено 18846 записей
Пример данных:
                                                text   category source
0  From: pef1@quads.uchicago.edu (it's enrico pal...  sci.space  train
1  From: jbh55289@uxa.cso.uiuc.edu (Josh Hopkins)...  sci.space  train
2  From: mrw9e@fulton.seas.Virginia.EDU (Michael ...  sci.space  train
3  From: dietz@cs.rochester.edu (Paul Dietz)\nSub...  sci.space  train
4  From: aws@iti.org (Allen W. Sherzer)\nSubject:...  sci.space  train
Загружено 18846 записей
Пример данных:
                                                text   category source
0  From: pef1@quads.uchicago.edu (it's enrico pal...  sci.space  train
1  From: jbh55289@uxa.cso.uiuc.edu (Josh Hopkins)...  sci.space  train
2  From: mrw9e@fulton.seas.Virginia.EDU (Michael ...  sci.space  train
3  From: dietz@cs.rochester.edu (Paul Dietz)\nSub...  sci.space  train
4  From: aws@iti.org (Allen W. Sherzer)\nSubject:...  sci.space  train

Предобработка текстов...

Анализ данных:
Количество категорий: 20
Рас

Источники:

http://qwone.com/~jason/20Newsgroups/ - 20news-bydate.tar.gz - 20 Newsgroups sorted by date; duplicates and some headers removed (18846 documents)